In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

In [ ]:
# Include all packages
import gc
import cv2
import shutil
import numpy as np
import pandas as pd
from time import time
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from yolov5.models.yolo import Model
from sklearn.model_selection import train_test_split

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import zipfile
    with zipfile.ZipFile('/content/drive/MyDrive/DL Project/DataSet1.zip', 'r') as zip_ref:
        zip_ref.extractall('./DataSet1')
except:
    print("Using Local Machine")

In [ ]:
# def CannyEdge(capturedImage):
#     grayScale = cv2.cvtColor(capturedImage, cv2.COLOR_BGR2GRAY)
#     constrastKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5) )
#     topHat = cv2.morphologyEx(grayScale, cv2.MORPH_TOPHAT, constrastKernel)
#     blackHat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, constrastKernel)
#     grayScale = grayScale + topHat - blackHat
#     bilateralFilter = cv2.bilateralFilter(grayScale, 11, 17, 17)
#     imageMedian = np.median(capturedImage)
#     lowerThreshold = max(0, (0.7 * imageMedian))
#     upperThreshold = min(255, (0.7 * imageMedian))
#     cannyEdgeImage = cv2.Canny(bilateralFilter, lowerThreshold, upperThreshold)
#     cannyEdgeImage = cv2.bitwise_not(cannyEdgeImage)
#     cannyEdgeImage = cv2.cvtColor(cannyEdgeImage, cv2.COLOR_GRAY2BGR)
#     return cannyEdgeImage



# GPT's SUGGESTIONS :

def CannyEdge(capturedImage):
    grayScale = cv2.cvtColor(capturedImage, cv2.COLOR_BGR2GRAY)
    
    # Optional: Apply a faster denoising method if needed.
    # grayScale = cv2.GaussianBlur(grayScale, (3, 3), 0)
    
    imageMedian = np.median(capturedImage)
    lowerThreshold = max(0, (0.7 * imageMedian))
    upperThreshold = min(255, (0.7 * imageMedian))
    cannyEdgeImage = cv2.Canny(grayScale, lowerThreshold, upperThreshold)
    cannyEdgeImage = cv2.bitwise_not(cannyEdgeImage)
    
    return cannyEdgeImage

def ResizeImage(image: np.ndarray, x1: int, y1: int, x2: int, y2: int, newWidth: int, newHeight: int) -> tuple:
    originalHeight, originalWidth = image.shape[:2]
    widthScale = newWidth / originalWidth
    heightScale = newHeight / originalHeight
    resizedImage = cv2.resize(
        image, (newWidth, newHeight), interpolation=cv2.INTER_LINEAR)
    x1New, y1New = int(x1 * widthScale), int(y1 * heightScale)
    x2New, y2New = int(x2 * widthScale), int(y2 * heightScale)
    return resizedImage, x1New, y1New, x2New, y2New


def LoadDataSet(dataSetFolderPath: str) -> tuple:
    images = []
    annotations = []
    annotationsFilePath = dataSetFolderPath+"/annotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=",")
    uniqueSigns = annotationsDataFrame['class'].unique().tolist()
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        images.append(image)
        annotations.append(
            [row[1], row[2], row[3], row[4]])

    del annotationsDataFrame

    return images, annotations, len(uniqueSigns)


def PreProcessDataSet(images: list, annotations: list, batchSize: int, resize: tuple) -> tuple:
    resizedImages = []
    newAnnotations = []
    for i, image in enumerate(images):
        [x1, y1, x2, y2] = annotations[i]
        resizedImage, x1New, y1New, x2New, y2New = ResizeImage(
            image, x1, y1, x2, y2, resize[0], resize[1])
        resizedImage = CannyEdge(resizedImage)
        resizedImages.append(resizedImage)
        newAnnotations.append(
            [x1New, y1New, x2New, y2New])

    X_train, X_val, y_train, y_val = train_test_split(
        resizedImages, newAnnotations, test_size=0.2, random_state=42)

    return X_train, X_val, y_train, y_val


In [ ]:
# GPT's SUGGESTIONS:
# When resizing the image, you can directly use the cv2.resize() function without calculating the width and height scale separately. This simplifies the code and makes it more readable.
# The ResizeImage function can be modified to accept the bounding box as a tuple, which makes the function call cleaner.
# In the LoadDataSet function, consider using a more efficient way of reading and appending images and annotations, such as list comprehensions.
# In the PreProcessDataSet function, it might be better to apply the CannyEdge function after splitting the data into training and validation sets. This way, you only process the images that will be used for training or validation.


def ResizeImage(image: np.ndarray, bbox: tuple, newWidth: int, newHeight: int) -> tuple:
    originalHeight, originalWidth = image.shape[:2]
    resizedImage = cv2.resize(image, (newWidth, newHeight), interpolation=cv2.INTER_LINEAR)
    x1New, y1New = int(bbox[0] * newWidth / originalWidth), int(bbox[1] * newHeight / originalHeight)
    x2New, y2New = int(bbox[2] * newWidth / originalWidth), int(bbox[3] * newHeight / originalHeight)
    return resizedImage, (x1New, y1New, x2New, y2New)

def LoadDataSet(dataSetFolderPath: str) -> tuple:
    annotationsFilePath = dataSetFolderPath+"/annotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=",")
    uniqueSigns = annotationsDataFrame['class'].unique().tolist()
    
    images = [cv2.imread(dataSetFolderPath+"/"+row[0]) for _, row in annotationsDataFrame[1:].iterrows()]
    annotations = [list(map(int, row[1:5])) for _, row in annotationsDataFrame[1:].iterrows()]

    del annotationsDataFrame

    return images, annotations, len(uniqueSigns)

def PreProcessDataSet(images: list, annotations: list, batchSize: int, resize: tuple) -> tuple:
    resizedImages, newAnnotations = zip(*[ResizeImage(image, bbox, resize[0], resize[1]) for image, bbox in zip(images, annotations)])
    
    X_train, X_val, y_train, y_val = train_test_split(resizedImages, newAnnotations, test_size=0.2, random_state=42)

    X_train = [CannyEdge(image) for image in X_train]
    X_val = [CannyEdge(image) for image in X_val]

    return X_train, X_val, y_train, y_val


class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputData, label = self.data[idx]

        if self.transform:
            inputData = self.transform(inputData)
        inputData = torch.from_numpy(inputData).float()
        label = torch.tensor(label).float()
        return inputData, label

def CreateDataLoaders(X_train, X_val, y_train, y_val, batchSize):
    trainDataSet = []
    valDataSet = []
    for i in range(len(X_train)):
        trainDataSet.append((X_train[i], y_train[i]))

    for i in range(len(X_val)):
        valDataSet.append((X_val[i], y_val[i]))

    trainDataSet = CustomDataset(trainDataSet)
    valDataSet = CustomDataset(valDataSet)
    trainDataLoader = DataLoader(
        trainDataSet, batch_size=batchSize, shuffle=True, num_workers=4)
    valDataLoader = DataLoader(
        valDataSet, batch_size=batchSize, shuffle=False, num_workers=4)

    return trainDataLoader, valDataLoader


In [ ]:
# GPT's SUGGESTIONS :
# Simplified the creation of trainDataSet and valDataSet by using the zip() function and converting the zipped object to a list. This makes the code more concise and easier to read.
# Removed the unnecessary for loop when creating the train and validation datasets.
# Replaced torch.from_numpy() with torch.tensor() to directly create a tensor from the input data. This makes the code more consistent.
# Added the pin_memory=True argument to both the trainDataLoader and valDataLoader. This allows faster data transfer between the CPU and GPU, which can improve training performance when using a GPU.

# Custom Dataset class that inherits from torch.utils.data.Dataset
class CustomDataset(Dataset):
    # Initialize the CustomDataset class with data and transform
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    # Define the length method to return the length of the dataset
    def __len__(self):
        return len(self.data)

    # Define the getitem method to return input data and labels as tensors
    def __getitem__(self, idx):
        inputData, label = self.data[idx]

        # Apply transform if provided
        if self.transform:
            inputData = self.transform(inputData)

        # Convert inputData to a PyTorch tensor
        inputData = torch.tensor(inputData, dtype=torch.float32)

        # Convert label to a PyTorch tensor
        label = torch.tensor(label, dtype=torch.float32)

        return inputData, label

# Function to create DataLoaders for train and validation datasets
def CreateDataLoaders(X_train, X_val, y_train, y_val, batchSize):
    # Combine input data and labels into a single list of tuples for train and validation datasets
    trainDataSet = list(zip(X_train, y_train))
    valDataSet = list(zip(X_val, y_val))

    # Create CustomDataset objects for train and validation datasets
    trainDataSet = CustomDataset(trainDataSet)
    valDataSet = CustomDataset(valDataSet)

    # Create DataLoaders for train and validation datasets
    trainDataLoader = DataLoader(trainDataSet, batch_size=batchSize, shuffle=True, num_workers=4, pin_memory=True)
    valDataLoader = DataLoader(valDataSet, batch_size=batchSize, shuffle=False, num_workers=4, pin_memory=True)

    return trainDataLoader, valDataLoader



def TargetstoTensors(targets, batchSize, numAnchors, gridSizes):
    targetObj = []
    targetBox = []
    for grid_size in gridSizes:
        targetObj.append(torch.zeros((batchSize, numAnchors, grid_size, grid_size, 1)))
        targetBox.append(torch.zeros((batchSize, numAnchors, grid_size, grid_size, 4)))

    for batch_index, target in enumerate(targets):
        x1, y1, x2, y2 = target.long()
        x_center, y_center, width, height = (x1 + x2) / 2, (y1 + y2) / 2, x2 - x1, y2 - y1

        for i, grid_size in enumerate(gridSizes):
            x_cell, y_cell = int(x_center * grid_size), int(y_center * grid_size)
            anchor = 0
            try:
                targetObj[i][batch_index, anchor, y_cell, x_cell, 0] = 1
                targetBox[i][batch_index, anchor, y_cell, x_cell] = torch.tensor([x_center, y_center, width, height])
            except Exception as e:
                pass
    return targetObj, targetBox


class SignboardLoss(nn.Module):
    def __init__(self, num_anchors=3):
        super(SignboardLoss, self).__init__()
        self.num_anchors = num_anchors

    def forward(self, preds, targets):
        objectLoss = torch.tensor(0.0, device=preds[0].device)
        boxLoss = torch.tensor(0.0, device=preds[0].device)
        batchSize = preds[0].size(0)
        gridSizes = [pred.size(2) for pred in preds]
        targetObjList, targetBoxList = TargetstoTensors(targets, batchSize, self.num_anchors, gridSizes)

        for i, pred in enumerate(preds):
            targetObj = targetObjList[i].to(pred.device)
            targetBox = targetBoxList[i].to(pred.device)

            objectLoss += nn.BCEWithLogitsLoss()(pred[..., 4:5], targetObj)
            boxLoss += nn.MSELoss()(pred[..., :4], targetBox)

        total_loss = objectLoss + boxLoss
        return total_loss

def CreateYolov5Model(numClasses: int, version: str):
    congfigFile = "yolov5/models/yolov5{}.yaml".format(version)
    model = Model(congfigFile, ch=3, nc=numClasses)
    # model.load_state_dict(torch.load("yolov5{}.pt".format(version))["model"].state_dict(), strict=False)

    return model


In [ ]:
# GPT's suggestions:
# Use a more suitable anchor selection method to determine the best anchor for each ground truth box. One approach could be to calculate the Intersection over Union (IoU) between the ground truth box and the anchors and assign the anchor with the highest IoU.
# Use a better loss function for the bounding box regression. The Mean Squared Error loss might not be the best option. Instead, you can use the Smooth L1 Loss or the IoU loss, which are more robust to outliers.
# Normalize the target bounding box coordinates to be in the range [0, 1]. This helps in training the model.
# You'll need to define the anchors variable and implement the calculate_iou function. The anchors variable should store the pre-defined anchor boxes for each grid size. The calculate_iou function should compute the Intersection over Union (IoU) between two bounding boxes


def TargetstoTensors(targets, batchSize, numAnchors, gridSizes, anchors):
    targetObj = []
    targetBox = []
    for grid_size in gridSizes:
        targetObj.append(torch.zeros((batchSize, numAnchors, grid_size, grid_size, 1)))
        targetBox.append(torch.zeros((batchSize, numAnchors, grid_size, grid_size, 4)))

    for batch_index, target in enumerate(targets):
        x1, y1, x2, y2 = target.long()
        x_center, y_center, width, height = (x1 + x2) / 2, (y1 + y2) / 2, x2 - x1, y2 - y1

        for i, grid_size in enumerate(gridSizes):
            x_cell, y_cell = int(x_center * grid_size), int(y_center * grid_size)
            anchor = 0

            # Calculate the IoUs between the ground truth box and the anchors
            ious = []
            for a in anchors[i]:
                iou = calculate_iou([x_center, y_center, width, height], a)
                ious.append(iou)
            anchor = torch.argmax(torch.tensor(ious))

            try:
                targetObj[i][batch_index, anchor, y_cell, x_cell, 0] = 1
                targetBox[i][batch_index, anchor, y_cell, x_cell] = torch.tensor([x_center, y_center, width, height]) / grid_size
            except Exception as e:
                pass
    return targetObj, targetBox

class SignboardLoss(nn.Module):
    def __init__(self, num_anchors=3):
        super(SignboardLoss, self).__init__()
        self.num_anchors = num_anchors

    def forward(self, preds, targets):
        objectLoss = torch.tensor(0.0, device=preds[0].device)
        boxLoss = torch.tensor(0.0, device=preds[0].device)
        batchSize = preds[0].size(0)
        gridSizes = [pred.size(2) for pred in preds]
        targetObjList, targetBoxList = TargetstoTensors(targets, batchSize, self.num_anchors, gridSizes, anchors)

        for i, pred in enumerate(preds):
            targetObj = targetObjList[i].to(pred.device)
            targetBox = targetBoxList[i].to(pred.device)

            objectLoss += nn.BCEWithLogitsLoss()(pred[..., 4:5], targetObj)
            boxLoss += nn.SmoothL1Loss()(pred[..., :4], targetBox)  # Use Smooth L1 Loss instead of MSE

        total_loss = objectLoss + boxLoss
        return total_loss

# ... other parts of the code
def calculate_iou(box1, box2):
    x1_center, y1_center, width1, height1 = box1
    width2, height2 = box2

    x1, y1 = x1_center - width1 / 2, y1_center - height1 / 2
    x2, y2 = x1_center + width1 / 2, y1_center + height1 / 2
    x3, y3 = x1_center - width2 / 2, y1_center - height2 / 2
    x4, y4 = x1_center + width2 / 2, y1_center + height2 / 2

    x_intersection = max(0, min(x2, x4) - max(x1, x3))
    y_intersection = max(0, min(y2, y4) - max(y1, y3))

    intersection_area = x_intersection * y_intersection
    box1_area = width1 * height1
    box2_area = width2 * height2
    union_area = box1_area + box2_area - intersection_area

    iou = intersection_area / union_area
    return iou


# NEED TO WORK ON THIS :

# Example of values for 'anchors' variable
anchors = [
    # Anchors for grid size 13x13
    [
        [1.08, 1.19],
        [3.42, 4.41],
        [6.63, 11.38]
    ],
    # Anchors for grid size 26x26
    [
        [0.54, 0.58],
        [1.55, 1.60],
        [3.42, 3.41]
    ],
    # Anchors for grid size 52x52
    [
        [0.27, 0.32],
        [0.62, 0.78],
        [1.70, 1.97]
    ]
]
# DISCLAIMER : You may need to adjust the anchor values to better suit your specific problem and dataset. The above example is just for illustration purposes. With these changes, your model should be better suited to handle the object detection task and potentially achieve higher accuracy and performance.

In [ ]:
# Calculating anchor values :


# To find the best anchor values for your dataset, you can use a technique called k-means clustering. The idea is to cluster the aspect ratios of the ground truth bounding boxes in your dataset and use the centroids of the clusters as the anchor box dimensions. Here's how to do it:
# Collect the ground truth bounding boxes from your dataset.
# Normalize the bounding box dimensions (width and height) by the input image dimensions (width and height). This will give you the normalized aspect ratios of the bounding boxes.
# Apply k-means clustering on the normalized aspect ratios to find the centroids. The number of clusters (k) should be equal to the number of anchor boxes you want to use. You can use a library like scikit-learn to perform k-means clustering.
# The resulting centroids are your optimal anchor box dimensions.
# Here's a sample code snippet to find the optimal anchor values using k-means clustering:
import numpy as np
from sklearn.cluster import KMeans

def get_anchor_values(annotations, num_anchors):
    aspect_ratios = []

    for annotation in annotations:
        x1, y1, x2, y2 = annotation
        width, height = x2 - x1, y2 - y1
        aspect_ratios.append([width, height])

    aspect_ratios = np.array(aspect_ratios)

    kmeans = KMeans(n_clusters=num_anchors, random_state=0)
    kmeans.fit(aspect_ratios)

    anchor_values = kmeans.cluster_centers_

    return anchor_values.tolist()

# In this function, annotations should be a list of bounding box coordinates (x1, y1, x2, y2), and num_anchors should be the total number of anchor boxes you want to use. This function will return a list of anchor values (width and height) that are best suited for your dataset.
# Keep in mind that the anchor values provided by this method will be for a single grid size. If you're using YOLO with multiple grid sizes (e.g., 13x13, 26x26, 52x52), you might need to perform k-means clustering for each grid size separately or adapt the clustering process to handle multiple grid sizes.

def TrainEpoch(model, dataLoader, optimizer, lossFunction, device):
    print("Training Epoch")
    model.train()
    runningLoss = 0
    dataLoaderLen = len(dataLoader)
    for i, (inputs, targets) in enumerate(dataLoader):
        # inputs = inputs.permute(2, 0, 1)
        inputs = inputs.permute(0, 3, 1, 2)
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = lossFunction(outputs, targets)
            loss.backward()
            optimizer.step()

        runningLoss += loss.item() * inputs.size(0)
        if(((i*100)//dataLoaderLen) % 10 == 0):
            print((i*100//dataLoaderLen), end="%,")
    epochLoss = runningLoss / dataLoaderLen
    return model, epochLoss


def ValidateEpoch(model, dataLoader, lossFunction, device):
    print("Validating Epoch")
    model.eval()
    runningLoss = 0
    dataLoaderLen = len(dataLoader)
    for i, (inputs, targets) in enumerate(dataLoader):
        inputs = inputs.permute(2, 0, 1)
        inputs = inputs.to(device)
        targets = targets.to(device)
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = lossFunction(outputs, targets)
        runningLoss += loss.item() * inputs.size(0)
        if(((i*100)//dataLoaderLen) % 10 == 0):
            print((i*100//dataLoaderLen), end="%,")
    epochLoss = runningLoss / dataLoaderLen
    return epochLoss



def TrainModel(model, trainDataLoader, valDataLoader, epochs, optimizer, scheduler, lossFunction, device):
    for epoch in range(epochs):
        startTime = time()
        print("Epoch {}/{}:".format(epoch+1, epochs))
        startTime = time()
        model, trainingEpochLoss = TrainEpoch(model, trainDataLoader, optimizer, lossFunction, device)
        # validationEpochLoss = ValidateEpoch(model, valDataLoader, lossFunction, device)
        # scheduler.step(validationEpochLoss)
        scheduler.step(trainingEpochLoss)
        endTime = time()
        timeTaken = endTime - startTime
        print()
        print("Training Loss: {:.4f}".format(trainingEpochLoss))
        # print("validation Loss: {:.4f}".format(validationEpochLoss))
        print("Time taken: {}min, {}, secs".format(timeTaken//60, int(timeTaken % 60)))
    
    print("Training complete.")
    return model


In [ ]:
# GPT's SUGGESTIONS:

# Add Tensorboard logging for better visualization of the training process.
# Implement early stopping to avoid overfitting and save time.
# Save the best model checkpoint based on the validation loss.
# Use learning rate scheduling to adapt the learning rate during training.


import torch
import numpy as np
from time import time
from torch.utils.tensorboard import SummaryWriter

def TrainEpoch(model, dataLoader, optimizer, lossFunction, device, writer, epoch):
    print("Training Epoch")
    model.train()
    runningLoss = 0
    dataLoaderLen = len(dataLoader)
    for i, (inputs, targets) in enumerate(dataLoader):
        inputs = inputs.permute(0, 3, 1, 2)
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = lossFunction(outputs, targets)
            loss.backward()
            optimizer.step()

        runningLoss += loss.item() * inputs.size(0)
        if(((i*100)//dataLoaderLen) % 10 == 0):
            print((i*100//dataLoaderLen), end="%,")
    
    epochLoss = runningLoss / dataLoaderLen
    writer.add_scalar('Loss/train', epochLoss, epoch)
    return model, epochLoss

def ValidateEpoch(model, dataLoader, lossFunction, device, writer, epoch):
    print("Validating Epoch")
    model.eval()
    runningLoss = 0
    dataLoaderLen = len(dataLoader)
    for i, (inputs, targets) in enumerate(dataLoader):
        inputs = inputs.permute(0, 3, 1, 2)
        inputs = inputs.to(device)
        targets = targets.to(device)
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = lossFunction(outputs, targets)
        runningLoss += loss.item() * inputs.size(0)
        if(((i*100)//dataLoaderLen) % 10 == 0):
            print((i*100//dataLoaderLen), end="%,")
    
    epochLoss = runningLoss / dataLoaderLen
    writer.add_scalar('Loss/val', epochLoss, epoch)
    return epochLoss

def TrainModel(model, trainDataLoader, valDataLoader, epochs, optimizer, scheduler, lossFunction, device, log_dir='runs', early_stopping_patience=10):
    writer = SummaryWriter(log_dir=log_dir)
    best_val_loss = np.inf
    epochs_without_improvement = 0
    best_model = None

    for epoch in range(epochs):
        startTime = time()
        print("Epoch {}/{}:".format(epoch+1, epochs))
        startTime = time()
        model, trainingEpochLoss = TrainEpoch(model, trainDataLoader, optimizer, lossFunction, device, writer, epoch)
        validationEpochLoss = ValidateEpoch(model, valDataLoader, lossFunction, device, writer, epoch)
        scheduler.step(validationEpochLoss)
        endTime = time()
        timeTaken = endTime - startTime

        if validationEpochLoss < best_val_loss:
            best_val_loss = validationEpochLoss
            best_model = model.state_dict()
            torch.save(best_model, 'best_model.pt')
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        print()
        print("Training Loss: {:.4f}".format(trainingEpochLoss))
        print("validation Loss: {:.4f}".format(validationEpochLoss))
        print("Time taken: {}min, {} secs".format(timeTaken // 60, int(timeTaken % 60)))

        if epochs_without_improvement >= early_stopping_patience:
            print("Early stopping triggered. No improvement in validation loss for {} epochs.".format(early_stopping_patience))
            break

    print("Training complete.")
    model.load_state_dict(best_model)
    return model


# Tensorboard logging is added to visualize the training and validation loss curves.
# Early stopping is implemented to halt training when there's no improvement in the validation loss for a given number of epochs (controlled by early_stopping_patience).
# The best model checkpoint (lowest validation loss) is saved to a file called 'best_model.pt'.
# Learning rate scheduling is already present in the original code (with scheduler.step(validationEpochLoss)).


batchSize = 32
inputShape = (640, 640)
epochs = 300
numAnchors = 3
yolo5Version = 'm'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Using {} device".format(device))

In [ ]:
# print("Downloading Weights of yolo5 Verion ", yolo5Version)
# weightsURL = "https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5{}.pt".format(yolo5Version)
# !wget {weightsURL}

images, annotations, numClasses = LoadDataSet("./DataSet1")
numClasses = 1

X_train, X_val, y_train, y_val = PreProcessDataSet(
    images, annotations, batchSize, inputShape)
del images
del annotations
gc.collect()

try:
    from google.colab.patches import cv2_imshow
    cv2_imshow(X_train[10])
except:
    print("using Local")
    # cv2.imshow("Input Image", image)

trainDataLoader, valDataLoader = CreateDataLoaders(
    X_train, X_val, y_train, y_val, batchSize)
del X_train
del y_train
del X_val
del y_val
gc.collect()

yolov5Model = CreateYolov5Model(numClasses,yolo5Version)
optimizer = optim.Adam(yolov5Model.parameters(), lr=0.001)
yolov5LossFunction= SignboardLoss()
yolov5Model = yolov5Model.to(device)
yolov5LossFunction = yolov5LossFunction.to(device)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)


trainedModel = TrainModel(yolov5Model, trainDataLoader,valDataLoader, epochs, optimizer, scheduler, yolov5LossFunction, device)

date = datetime.now()
date = date.strftime("%m-%d-%H")
torch.save(trainedModel.state_dict(), 'yolov5Modelv2' + date +'.pth')
shutil.copy('/content/yolov5Modelv2' + date +'.pth', '/content/drive/MyDrive/DL Project/Trained Models/yolov5Modelv2' + date +'.pth')


In [ ]:
# GPT'S SUGGESTIONS :
# Removed the gc.collect() calls as they are not necessary in most cases. The garbage collector will automatically clean up unreferenced objects.
# Added early_stopping_patience parameter to the TrainModel function to implement early stopping.
# Added Tensorboard logging to the TrainModel function to visualize training and validation losses.


import torch
import gc
from torch.optim.lr_scheduler import ReduceLROnPlateau
import shutil
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

batchSize = 32
inputShape = (640, 640)
epochs = 300
numAnchors = 3
yolo5Version = 'm'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using {} device".format(device))

images, annotations, numClasses = LoadDataSet("./DataSet1")
numClasses = 1
X_train, X_val, y_train, y_val = PreProcessDataSet(images, annotations, batchSize, inputShape)

trainDataLoader, valDataLoader = CreateDataLoaders(X_train, X_val, y_train, y_val, batchSize)

yolov5Model = CreateYolov5Model(numClasses, yolo5Version)
optimizer = optim.Adam(yolov5Model.parameters(), lr=0.001)
yolov5LossFunction = SignboardLoss()
yolov5Model = yolov5Model.to(device)
yolov5LossFunction = yolov5LossFunction.to(device)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

# Set up TensorBoard logging
writer = SummaryWriter()

trainedModel = TrainModel(yolov5Model, trainDataLoader, valDataLoader, epochs, optimizer, scheduler, yolov5LossFunction, device, writer, early_stopping_patience=20)

date = datetime.now()
date = date.strftime("%m-%d-%H")
torch.save(trainedModel.state_dict(), 'yolov5Modelv2' + date + '.pth')
shutil.copy('/content/yolov5Modelv2' + date + '.pth', '/content/drive/MyDrive/DL Project/Trained Models/yolov5Modelv2' + date + '.pth')
